In [6]:
import pandas as pd
import numpy as np
from pct.tree.tree import Tree
from pct.tree.heuristic.Heuristic import Heuristic5
from pct.tree.heuristic.NumericHeuristic import NumericHeuristic5
from pct.tree.splitter.splitter import Splitter
from pct.tree.ftest.ftest import FTest




In [8]:
from sklearn.preprocessing import LabelEncoder

# Load the u.data dataset
u_data = pd.read_csv('C:/Users/13447/Desktop/Stats thesis/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

# Perform label encoding on user_id and item_id
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

u_data['user_id'] = user_encoder.fit_transform(u_data['user_id'])
u_data['item_id'] = item_encoder.fit_transform(u_data['item_id'])

# Create the rating matrix
n_users = u_data['user_id'].nunique()
n_items = u_data['item_id'].nunique()

# Define the threshold
THRESHOLD = 3  # Ratings above this are "Lovers", below are "Haters"

# Create a user-item matrix with users as rows and items as columns, and fill missing values with 0
rating_matrix = u_data.pivot(index='user_id', columns='item_id', values='rating')
rating_matrix.index = u_data['user_id'].unique()  # Ensure user_id alignment
rating_matrix.columns = u_data['item_id'].unique()  # Ensure item_id alignment


# Apply threshold classification to the rating matrix (after thresholding)
rating_matrix_thresholded = np.where(rating_matrix > THRESHOLD, 1, np.where(rating_matrix > 0, -1, 0))

# Convert numpy array to pandas DataFrame for easier inspection
rating_matrix_thresholded_df = pd.DataFrame(rating_matrix_thresholded)


# Create rI and rU indexes for training set (R)
# rI: item to user ratings (dictionary of items with lists of user ratings)
rI = {}
for _, row in u_data.iterrows():
    item_id = row['item_id']
    user_id = row['user_id']
    rating = row['rating']
    if item_id not in rI:
        rI[item_id] = []
    rI[item_id].append((user_id, rating))

# rU: user to item ratings (dictionary of users with lists of item ratings)
rU = {}
for _, row in u_data.iterrows():
    user_id = row['user_id']
    item_id = row['item_id']
    rating = row['rating']
    if user_id not in rU:
        rU[user_id] = []
    rU[user_id].append((item_id, rating))

In [17]:
assert set(rI.keys()).issubset(set(rating_matrix.columns)), "Mismatch in rI keys!"
assert set(rU.keys()).issubset(set(rating_matrix.index)), "Mismatch in rU keys!"


In [ ]:
# find and count users who have rated item 246 and their ratings
item_id = 55
print(f"Users who have rated item {item_id} and their ratings:")
print(rI[item_id])

# count users who have rated item 246 and their ratings
print(f"Number of users who have rated item {item_id}: {len(rI[item_id])}")

In [13]:

print(f"Number of users who didn't rate item {item_id}:")
print(n_users - len(rI[item_id]))

Number of users who didn't rate item 55:
549


In [9]:
rating_matrix

,241,301,376,50,345,473,264,464,450,85,...,1593,1625,1644,1658,1681,1673,1639,1636,1629,1640
195,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
929,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# rating_matrix_thresholded_df.head()

In [ ]:
# Optionally, print rI and rU to check the data structure 
print("\nSample rI (item to user ratings) for Training Set:")
for item in list(rI.keys())[:2]:  # Display sample item indices
    print(f"Item {item}: {rI[item]}")

print("\nSample rU (user to item ratings) for Training Set:")
for user in list(rU.keys())[:2]:  # Display sample user indices
    print(f"User {user}: {rU[user]}")

## initiate tree and splitter

In [12]:
tree = Tree(min_instances=6, max_depth=6)
tree.fit(rating_matrix, u_data['rating'], target_weights=None, rI=rI, rU=rU)




✅ Converted x and y to DataFrame
✅ Assigned x and y
✅ Creating target weights...
✅ Identifying numerical and categorical attributes...
✅ Creating Splitter...
Initializing Splitter...
✅ Calling build()...
🌲 Building predictive clustering tree...


C:\Users\13447\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  791
👥 Users who rated item 791: 86
❤️ Lovers: 59
💔 Haters: 27
❓ Unknowns: 857
🔄 Recursively building tree for subsets...
🌲 Building predictive clustering tree...


C:\Users\13447\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  689
👥 Users who rated item 689: 155
❤️ Lovers: 84
💔 Haters: 71
❓ Unknowns: 52
🔄 Recursively building tree for subsets...
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...


...

🔍 Best item for splitting:  791
👥 Users who rated item 791: 86
❤️ Lovers: 59
💔 Haters: 27
❓ Unknowns: 499
🔄 Recursively building tree for subsets...
🌲 Building predictive clustering tree...


C:\Users\13447\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  422
👥 Users who rated item 422: 300
❤️ Lovers: 199
💔 Haters: 101
❓ Unknowns: 31
🔄 Recursively building tree for subsets...
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x00000222980B4210>. Stopping recursion.
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x00000222980B4210>. Stopping recursion.
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x00000222980B4210>. Stopping recursion.
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  517
👥 Users who rated item 517: 89
❤️ Lovers: 59
💔 Haters: 30
❓ Unknowns: 14
🔄 Recursively building tree for subsets...
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x000002229CEBD4D0>. Stopping recursion.
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x000002229CEBD4D0>. Stopping recursion.
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x000002229CEBD4D0>. Stopping recursion.
🌲 Building predictive clustering tre

C:\Users\13447\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
C:\Users\13447\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  791
👥 Users who rated item 791: 86
❤️ Lovers: 59
💔 Haters: 27
❓ Unknowns: 443
🔄 Recursively building tree for subsets...
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x000002229E6BE950>. Stopping recursion.
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x000002229E6BE950>. Stopping recursion.
🍃 Reached max depth at node <pct.tree.node.node.Node object at 0x000002229E6BE950>. Stopping recursion.
✅ Tree built successfully!


In [14]:
tree.print_tree_structure()

Node: 791_1
Attribute: 791
Value: [0.]
Criterion: None
Leaf Node: No
Children:
    Node: 689_2
    Attribute: 689
    Value: [0.]
    Criterion: <pct.tree.node.node.Node object at 0x0000022296565610>
    Leaf Node: No
    Children:
        Node: leaf_3=[2.6] (5.0)
        Attribute: None
        Value: None
        Criterion: None
        Leaf Node: No
        Children:
            Node: 791_1
            Attribute: 791
            Value: [0.]
            Criterion: None
            Leaf Node: No
            Children:
                Node: 689_2
                Attribute: 689
                Value: [0.]
                Criterion: <pct.tree.node.node.Node object at 0x0000022296565610>
                Leaf Node: No
                Children:
                    Node: leaf_3=[2.6] (5.0)
                    Attribute: None
                    Value: None
                    Criterion: None
                    Leaf Node: No
                    Children:
                        Node: 791_1
  